In [1]:
from utils import get_data_path, load_csv, load_shapefile

In [2]:
load_csv('ppa_fonte.csv')

,id,codigo orgao,descricao orgao,codigo unidade,descricao unidade,codigo da função,descricao da funcao,codigo subfuncao,descricao da subfuncao,codigo programa,descricao programa,codigo acao,descricao acao,codigo fonte,descricao fonte,' 2022',' 2023',' 2024',' 2025'
0,2_10_3011_2818,2,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",0,Tesouro Municipal,1501000.0,1729125.0,1746415.0,1764978.0
1,2_10_3024_2100,2,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,0,Tesouro Municipal,237584938.0,272179517.0,274852853.0,277723036.0
2,2_10_3011_1220,2,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,126,Tecnologia da Informação,3011,"Modernização Tecnológica, Desburocratização e ...",1220,Desenvolvimento de Sistemas de Informação e Co...,0,Tesouro Municipal,1000.0,0.0,0.0,0.0
3,2_10_3024_2171,2,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,126,Tecnologia da Informação,3024,Suporte Administrativo,2171,Manutenção e Operação de Sistemas de Informaçã...,0,Tesouro Municipal,3400000.0,3916743.0,3955910.0,3997962.0
4,2_10_3003_9306,2,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,302,Assistência Hospitalar e Ambulatorial,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",9306,E1019 - Compra de Equipamentos para o Hospital...,0,Tesouro Municipal,150000.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,99_10_3011_2818,99,Fundo Municipal de Iluminação Pública,10,Fundo Municipal de Iluminação Pública,15,Urbanismo,126,Tecnologia da Informação,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",8,Tesouro Municipal - Recursos Vinculados,200000.0,210000.0,311909.0,323513.0
2318,99_10_3024_2171,99,Fundo Municipal de Iluminação Pública,10,Fundo Municipal de Iluminação Pública,15,Urbanismo,126,Tecnologia da Informação,3024,Suporte Administrativo,2171,Manutenção e Operação de Sistemas de Informaçã...,8,Tesouro Municipal - Recursos Vinculados,1739720.0,1826706.0,2713168.0,2814114.0
2319,99_10_3022_5160,99,Fundo Municipal de Iluminação Pública,10,Fundo Municipal de Iluminação Pública,15,Urbanismo,451,Infra-Estrutura Urbana,3022,Requalificação e Promoção da Ocupação dos Espa...,5160,"Implantação, Ampliação e Requalificação da Red...",8,Tesouro Municipal - Recursos Vinculados,3000.0,3000.0,3000.0,3000.0
2320,99_10_3022_6027,99,Fundo Municipal de Iluminação Pública,10,Fundo Municipal de Iluminação Pública,15,Urbanismo,452,Serviços Urbanos,3022,Requalificação e Promoção da Ocupação dos Espa...,6027,Contraprestação de Parceria Público-Privada (P...,8,Tesouro Municipal - Recursos Vinculados,430315630.0,451825948.0,671088639.0,696057046.0


In [3]:
load_shapefile('subprefs/subprefs.shp')

,cd_identif,cd_subpref,nm_subpref,tx_escala,sg_fonte_o,dt_criacao,cd_tipo_di,dt_atualiz,cd_usuario,sg_subpref,qt_area_qu,qt_area_me,geometry
0,26.0,16,CIDADE ADEMAR,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,AD,31.0,3.081024e+07,"POLYGON ((332383.832 7381216.079, 332408.532 7..."
1,15.0,25,MOOCA,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,MO,36.0,3.604691e+07,"POLYGON ((334126.109 7396389.068, 334125.831 7..."
2,19.0,13,IPIRANGA,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,IP,37.0,3.749732e+07,"POLYGON ((335254.612 7391150.376, 335253.982 7..."
3,13.0,08,LAPA,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,LA,41.0,4.063870e+07,"POLYGON ((321633.729 7393535.365, 321633.031 7..."
4,28.0,15,JABAQUARA,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,JA,14.0,1.400981e+07,"POLYGON ((331544.24 7384735.403, 331562.36 738..."
5,12.0,28,GUAIANASES,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,GU,18.0,1.777556e+07,"POLYGON ((354697.574 7396050.701, 354696.418 7..."
6,29.0,14,SANTO AMARO,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,SA,38.0,3.774532e+07,"POLYGON ((327892.375 7386141.134, 327896.575 7..."
7,32.0,32,SAPOPEMBA,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,SB,14.0,1.363290e+07,"POLYGON ((344641.98 7387700.637, 344641.228 73..."
8,9.0,23,SAO MIGUEL,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,MP,26.0,2.615923e+07,"POLYGON ((352480.323 7397515.871, 352477.052 7..."
9,5.0,06,JACANA-TREMEMBE,1:5000,GEOGSG,2023-11-30,100200300,2025-07-05,None,JT,65.0,6.511566e+07,"POLYGON ((335167.648 7404409.048, 335167.247 7..."
